In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input
from keras.layers import Embedding, LSTM
from keras.models import Sequential

import numpy as np
import csv


Using TensorFlow backend.


In [2]:
GLOVE_PATH = ''
TRAIN_DATA_PATH = ''

SENTENCE_LENGTH = 100
EMBEDDING_DIM = 50
TEST_SPLIT = 0.2

# Tensorflow throws OOM if trying to use the entire glove embedding, we can reduce the embedding size only to data's input
# Embeddings that are not part of the input won't be used for training anyway
VOCAB_SIZE = 1000

In [3]:
def loadGlove(path):
    embeddings_index = {}
    with open(path, "r", encoding="utf8") as glovef:
        for line in glovef:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

In [4]:
def loadDataSet(path):
    label = []
    data = []

    with open(path,'r') as tsvin:
        tsvin = csv.reader(tsvin, delimiter='\t')
        for row in tsvin:
            label.append(int(row[0]))
            data.append(row[1])
    return data, label

In [5]:
def split(ratio, data, labels):
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    
    data = data[indices]
    labels = labels[indices]
    
    nb_test = int(ratio * data.shape[0])

    x_train = data[:-nb_test]
    y_train = labels[:-nb_test]

    x_test = data[-nb_test:]
    y_test = labels[-nb_test:]
    
    return x_train, y_train, x_test, y_test

In [6]:
embeddings_table = loadGlove(GLOVE_PATH)
data_lines, data_labels = loadDataSet(TRAIN_DATA_PATH)

print('Found %s training instance.' % len(data_lines))

Found 400000 word vectors.
Found 1574 training instance.


In [7]:
# should save the tokenizer so that the text sequence is consistant

tokenizer = Tokenizer(nb_words=VOCAB_SIZE)
tokenizer.fit_on_texts(data_lines)
data_sequences = tokenizer.texts_to_sequences(data_lines)
data_index = tokenizer.word_index
vocab_dim = min(VOCAB_SIZE, len(data_index))

In [8]:
data_padded = pad_sequences(data_sequences, maxlen=SENTENCE_LENGTH)
data_labels = np.asarray(data_labels)

In [9]:
x_train, y_train, x_test, y_test = split(TEST_SPLIT, data_padded, data_labels)

In [10]:
embedding_matrix = np.zeros((vocab_dim, EMBEDDING_DIM))

for word, i in data_index.items():
    if i < vocab_dim: 
        embedding_vector = embeddings_table.get(word)

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(vocab_dim, EMBEDDING_DIM, weights=[embedding_matrix], input_length=SENTENCE_LENGTH, trainable=False)

In [11]:
print('Training model.')

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2, return_sequences=True)) 
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test),nb_epoch=20, batch_size=20)


Training model.
Train on 1260 samples, validate on 314 samples
Epoch 1/20
1260/1260 [==============================] - 25s - loss: 0.6990 - acc: 0.5500 - val_loss: 0.6939 - val_acc: 0.5414
Epoch 2/20
1260/1260 [==============================] - 25s - loss: 0.6611 - acc: 0.5937 - val_loss: 0.6320 - val_acc: 0.6274
Epoch 3/20
1260/1260 [==============================] - 25s - loss: 0.6298 - acc: 0.6460 - val_loss: 0.6715 - val_acc: 0.6019
Epoch 4/20
1260/1260 [==============================] - 25s - loss: 0.6156 - acc: 0.6548 - val_loss: 0.6079 - val_acc: 0.6401
Epoch 5/20
1260/1260 [==============================] - 25s - loss: 0.5932 - acc: 0.6810 - val_loss: 0.5829 - val_acc: 0.7006
Epoch 6/20
1260/1260 [==============================] - 26s - loss: 0.5860 - acc: 0.6730 - val_loss: 0.5700 - val_acc: 0.7038
Epoch 7/20
1260/1260 [==============================] - 25s - loss: 0.5825 - acc: 0.6857 - val_loss: 0.5709 - val_acc: 0.6911
Epoch 8/20
1260/1260 [==============================] -